# Machine Learning Code - Training with real data.

In [ ]:
import os
import sys
import socket
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import imageio
import io
from IPython.display import display, Image as IPImage, clear_output
import importlib
import warnings
import pathlib
from pathlib import Path
import shutil

# Suppress warnings if needed
warnings.filterwarnings("ignore")

def find_src_directory(current_directory: Path) -> Path:
    # Loop through the parent directories
    for parent in current_directory.parents:
        potential_src = parent / 'src'
        if potential_src.is_dir():
            return potential_src
    return None
current_dir = pathlib.Path().absolute()
Advanced_Microscopy_dir = find_src_directory(current_dir)
sys.path.append(str(Advanced_Microscopy_dir))

# Local module imports
import Advanced_Microscopy as AM 
import ML_SpotDetection as ML

# Reload modules if they are under active development
importlib.reload(AM)
importlib.reload(ML)

# Load pre-trained model
model = ML.ParticleDetectionCNN()
model_path = 'particle_detection_cnn.pth'
ML.load_model(model, model_path)

# Utility functions
def find_src_directory(current_directory: Path) -> Path:
    """ Finds the 'src' directory by navigating up from the current directory. """
    for parent in current_directory.parents:
        potential_src = parent / 'src'
        if potential_src.is_dir():
            return potential_src
    return None

# Display computer info
computer_name = socket.gethostname()
computer_user_name = computer_name.split('.')[0]


In [ ]:
path_my_folder = pathlib.Path(r"/Users/nzlab-la/Library/CloudStorage/OneDrive-TheUniversityofColoradoDenver/General - Zhao (NZ) Lab/Microscope/Luis Aguilera/Live cell imaging_Folding & Nascent chains")
data_folder_path =path_my_folder.joinpath(r"20240806 pNZ212 and pRS001_JF646_NoDelay_ch0 folding_ch1 nascent chains.lif")
list_images, list_names, pixel_xy_um, voxel_z_um, channel_names, number_color_channels,list_time_intervals, bit_depth = AM.ReadLif(data_folder_path,show_metadata=False,save_tif=False,save_png=True,format='TZYXC').read()
selected_image = 4
real_image =list_images[selected_image]

In [ ]:
number_time_points = real_image.shape[0]
filtered_image_normalized = np.zeros_like(real_image)
for i in range(number_time_points):
    filtered_image_normalized[i] = AM.RemoveExtrema(real_image[i],min_percentile=0.5, max_percentile=99).remove_outliers() 

# renaming filtered_image_normalized as real_image
real_image = filtered_image_normalized

In [ ]:
batch_size = 256
num_epochs = batch_size*200
learning_rate = 0.000001
grid_size = 11
crop_size = grid_size


In [ ]:
results_folder = Path('results_'+data_folder_path.stem + '_cell_id_'+str(selected_image))
df_tracking = pd.read_csv(results_folder.joinpath('results_df_tracking.csv'))

In [ ]:
def create_crops_from_image(real_image,df_tracking=None, minimal_snr = 0.8, grid_size = 11, selected_color_channel = 0):
    
    if df_tracking is not None:
        df_tracking_selected = df_tracking[df_tracking['snr_ch_0'] >minimal_snr]
        time = np.random.randint(real_image.shape[0])
        frame_data = df_tracking_selected[df_tracking_selected['frame'] == time]
        number_spots = len(frame_data)
        selected_spot = np.random.randint(number_spots)
        positions = frame_data[['z', 'y', 'x']].to_numpy().astype(int)
        z = positions[selected_spot,0]
        y = positions[selected_spot,1]
        x = positions[selected_spot,2]
    else:
        time = np.random.randint(real_image.shape[0])
        x = np.random.randint(grid_size, real_image.shape[3] - grid_size)
        y = np.random.randint(grid_size, real_image.shape[2] - grid_size)
        z = np.random.randint(real_image.shape[1])
    crop = real_image[time, z, 
                    y-grid_size//2: y+grid_size//2 + 1,
                    x-grid_size//2: x+grid_size//2 + 1, selected_color_channel]
    if np.max(crop) > 0:
        crop= ((crop - np.min(crop)) / (np.max(crop) - np.min(crop))) * 255
    return crop

In [ ]:
# dataset folder
traning_dataset_folder_real_data = 'training_crops_real_data'
if os.path.exists(traning_dataset_folder_real_data):
    shutil.rmtree(traning_dataset_folder_real_data)
Path(traning_dataset_folder_real_data).mkdir(parents=True, exist_ok=True)




In [ ]:
# creating the dataset with 100 simulated single spots
number_simulated_single_spots = 1000
for i in range(number_simulated_single_spots):
    z = create_crops_from_image(real_image,df_tracking)
    im = Image.fromarray((z).astype(np.uint8))  # Normalize and convert to uint8
    im.save(f'{traning_dataset_folder_real_data}/particle_{i}.png')


# creating a dataset without spots start counter from 0 
number_simulated_no_spots = 1000
for i in range(number_simulated_no_spots):
    z = create_crops_from_image(real_image,df_tracking=None)
    im = Image.fromarray((z).astype(np.uint8))  # Normalize and convert to uint8
    im.save(f'{traning_dataset_folder_real_data}/no_particle_{i}.png')

In [ ]:
load_model_from_file = False
model_name_real_data = 'particle_detection_cnn_real_data.pth'
if load_model_from_file == False:
    importlib.reload(ML)
    model_real_data, training_losses_real_data, validation_losses_real_data = ML.run_network(image_dir=traning_dataset_folder_real_data, num_epochs=num_epochs,learning_rate = learning_rate,batch_size = batch_size)
    ML.save_model(model_real_data, path=model_name_real_data)
    plt.figure(figsize=(12, 6))
    plt.plot(training_losses_real_data, label='Training Loss', color='blue', marker='o', linestyle='-', linewidth=2, markersize=6)
    plt.plot(validation_losses_real_data, label='Validation Loss', color='red', marker='x', linestyle='--', linewidth=2, markersize=8)
    plt.title('Training and Validation Loss Over Epochs', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()  # Adjusts plot margins to give a better layout
    plt.show()
else:
    model_real_data = ML.ParticleDetectionCNN()
    model_path_real_data = model_name_real_data
    ML.load_model(model_real_data, model_path_real_data)

In [ ]:
# save training_losses_real_data, validation_losses_real_data as numpy arrays
np.save('training_losses_real_data.npy', training_losses_real_data)
np.save('validation_losses_real_data.npy', validation_losses_real_data)


# ML model with simulated data

____

In [ ]:
def plot_spot(amplitude=None, sigma=None, grid_size=11, mu_x=None, mu_y=None, percentage_noise=None, create_spot=False, number_spots=1,real_image= None,selected_color_channel=0):
    x = np.linspace(0, grid_size - 1, grid_size)
    y = np.linspace(0, grid_size - 1, grid_size)
    x, y = np.meshgrid(x, y)
    if real_image is not None:
        selected_z = np.random.randint(0, real_image.shape[1])
        selected_time = np.random.randint(0, real_image.shape[0])
        position_x = np.random.randint(grid_size, real_image.shape[3]- grid_size)
        position_y = np.random.randint(grid_size, real_image.shape[2]- grid_size)
        max_val_real_image = np.percentile(real_image[selected_time, selected_z, :, :, selected_color_channel], 100)
        z = real_image[selected_time, selected_z, position_y:position_y+grid_size, position_x:position_x+grid_size, selected_color_channel]
        basal_intensity = np.random.randint(100, 255//2)
        # min max normalization
        z = (z / max_val_real_image)*basal_intensity
        if amplitude is None:
            # random int between 10 and 100
            amplitude = np.random.rand()*np.random.randint(2,4) *basal_intensity
        else:
            amplitude = amplitude
    else:
        z = np.zeros((grid_size, grid_size))
    
    if create_spot == False:
        sigma = 6
        percentage_noise = 0.01 + np.random.rand() * 0.05
        mu_x, mu_y = (grid_size/2)-0.5, (grid_size/2)-0.5  # Default to center if not specified
        if real_image is None:
            spot_intensity = amplitude * np.exp(-((x - mu_x)**2 + (y - mu_y)**2) / (2 * sigma**2))
            z += spot_intensity
            if percentage_noise > 0:
                z = z + np.random.normal(0, amplitude*percentage_noise, z.shape)  
        # shuffle all the values in the image to ensure there is no spot 
        z = z.flatten()
        np.random.shuffle(z)
        z = z.reshape(grid_size, grid_size)
    else:
        for _ in range(number_spots):
            if percentage_noise is None:
                percentage_noise_temp = 0.01 + np.random.rand() * 0.05
            else:
                percentage_noise_temp = percentage_noise
            if amplitude is None:
                # random number between 200 and 255
                amplitude_temp = int( 200 + np.random.rand() * 55) 
            else:
                amplitude_temp = amplitude
            if sigma is None:
                # random number between 1 and 2.5
                sigma_temp = 0.5 + np.random.rand() * 1.5
            else:
                sigma_temp = sigma
            if mu_x is None or mu_y is None:
                # Generate random coordinates within the range 2 to 6
                # generate the random coordinates within the range of -3 to 3 from the center
                image_center = grid_size//2
                number_of_pixels_around = np.min([image_center-1, 4])
                mu_x_temp = image_center + np.random.randint(-number_of_pixels_around, number_of_pixels_around)
                mu_y_temp = image_center + np.random.randint(-number_of_pixels_around, number_of_pixels_around)
                #mu_x_temp = 3 + np.random.rand() * 3

                #mu_x_temp = 3 + np.random.rand() * 3
                #mu_y_temp = 3 + np.random.rand() * 3
                #mu_x_temp = 3 + np.random.rand() * 3
                #mu_y_temp = 3 + np.random.rand() * 3
            # Compute Gaussian intensity
            spot_intensity = amplitude_temp * np.exp(-((x - mu_x_temp)**2 + (y - mu_y_temp)**2) / (2 * sigma_temp**2))
            z += spot_intensity  # Add intensity to the spot image
        # adding noise to the image if not real image
        if real_image is None:
            amplitude = amplitude_temp
            percentage_noise = percentage_noise_temp
            if percentage_noise > 0:
                percentage_noise_temp = 0.01 + np.random.rand() * 0.1
                z = z + np.random.normal(0, amplitude * percentage_noise, z.shape)
    # Normalize and add noise if applicable
    if z.max() > 0:
        z = ((z - np.min(z)) / (np.max(z) - np.min(z))) * 255
    z = np.clip(z, 0, 255).astype(np.uint8)  # Ensure values are within uint8 range
    return z

In [ ]:
# dataset folder
traning_dataset_folder_simulated_data = 'training_crops_simulated_data'
# remove the folder if it exists
if os.path.exists(traning_dataset_folder_simulated_data):
    shutil.rmtree(traning_dataset_folder_simulated_data)
Path(traning_dataset_folder_simulated_data).mkdir(parents=True, exist_ok=True)

In [ ]:
mu_x = None  # Center x-coordinate 2-6
mu_y = None  # Center y-coordinate 2-6
amplitude = None
sigma = None
number_doubles = 256
# creating the dataset with 100 simulated single spots
number_simulated_single_spots = 500- number_doubles #2000
for i in range(number_simulated_single_spots):
    z = plot_spot(amplitude, sigma, grid_size, mu_x, mu_y, percentage_noise=None,create_spot=True,number_spots=1,real_image=real_image)
    im = Image.fromarray((z).astype(np.uint8))  # Normalize and convert to uint8
    im.save(f'{traning_dataset_folder_simulated_data}/particle_{i}.png')

# creating the dataset with 100 simulated double spots continue from the last index
number_simulated_double_spots = number_doubles
for i in range(number_simulated_double_spots):
    z = plot_spot(amplitude, sigma, grid_size, mu_x, mu_y, percentage_noise=None,create_spot=True,number_spots=2,real_image=real_image)
    im = Image.fromarray((z).astype(np.uint8))  # Normalize and convert to uint8
    im.save(f'{traning_dataset_folder_simulated_data}/particle_{i+number_simulated_single_spots}.png')

# adding the real data 
number_real_spots = 1000
for i in range(number_real_spots):
    z = create_crops_from_image(real_image,df_tracking)
    im = Image.fromarray((z).astype(np.uint8))  # Normalize and convert to uint8
    im.save(f'{traning_dataset_folder_simulated_data}/particle_{i+number_simulated_double_spots+number_simulated_single_spots}.png')

# creating a dataset without spots start counter from 0 
number_simulated_no_spots = 500 #2000
for i in range(number_simulated_no_spots):
    z = plot_spot(amplitude, sigma, grid_size, mu_x, mu_y, percentage_noise=None,create_spot=False,number_spots=0,real_image=real_image)
    im = Image.fromarray((z).astype(np.uint8))  # Normalize and convert to uint8
    im.save(f'{traning_dataset_folder_simulated_data}/no_particle_{i}.png')

# creating a dataset without spots start counter from 0 
number_real_no_spots = 500
for i in range(number_real_no_spots):
    z = create_crops_from_image(real_image,df_tracking=None)
    im = Image.fromarray((z).astype(np.uint8))  # Normalize and convert to uint8
    im.save(f'{traning_dataset_folder_simulated_data}/no_particle_{number_simulated_no_spots+i}.png')


# Model machine learning tranining with simulated data
____

In [ ]:
load_model_from_file = False
model_name_simulated_data = 'particle_detection_cnn_simulated_data.pth'
if load_model_from_file == False:
    importlib.reload(ML)
    model_simulated_data, training_losses_simulated_data, validation_losses_simulated_data = ML.run_network(image_dir=traning_dataset_folder_simulated_data, num_epochs=num_epochs,learning_rate = learning_rate,batch_size = batch_size)
    ML.save_model(model_simulated_data, path=model_name_simulated_data)
    plt.figure(figsize=(12, 6))
    plt.plot(training_losses_simulated_data, label='Training Loss', color='blue', marker='o', linestyle='-', linewidth=2, markersize=6)
    plt.plot(validation_losses_simulated_data, label='Validation Loss', color='red', marker='x', linestyle='--', linewidth=2, markersize=8)
    plt.title('Training and Validation Loss Over Epochs', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()  # Adjusts plot margins to give a better layout
    plt.show()
else:
    model_simulated_data = ML.ParticleDetectionCNN()
    ML.load_model(model_simulated_data, model_name_simulated_data)


In [ ]:
plt.plot(training_losses_simulated_data, label='Training Loss', color='blue',  linestyle='-', linewidth=2, markersize=6)
plt.plot(validation_losses_simulated_data, label='Validation Loss', color='red', linestyle='--', linewidth=2, markersize=8)
plt.title('Training and Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()  # Adjusts plot margins to give a better layout
# make y axis log scale
plt.yscale('log')
plt.show()

In [ ]:
# save training_losses_simulated_data, validation_losses_simulated_data as numpy arrays
np.save('training_losses_simulated_data.npy', training_losses_simulated_data)
np.save('validation_losses_simulated_data.npy', validation_losses_simulated_data)


# Training the model with user reinforced data.

___

In [ ]:
# loading a list of crops 
max_crops_to_display =50
selected_time_point = None
croparray_filtered, mean_crop_filtered, first_appearance, crop_size = AM.CropArray(image=filtered_image_normalized, df_crops=df_tracking, crop_size=crop_size, remove_outliers=False, max_percentile=99.9,selected_time_point=selected_time_point).run()
list_crops = AM.Utilities().normalize_crop_return_list(array_crops_YXC=mean_crop_filtered,crop_size=crop_size,selected_color_channel=0,normalize_to_255=True)

In [ ]:
# impot the human selection
a = np.load('human_selection_Jake.npy')
b = np.load('human_selection_Luis.npy')
c = np.load('human_selection_Rhiannon.npy')
#d = np.load('human_selection_Nate.npy')

flag_vector_ground_truth_all_true = np.logical_and.reduce([a, b, c])
flag_vector_ground_truth_all_false = np.logical_not(np.logical_or.reduce([a, b, c]))

In [ ]:
traning_dataset_folder_human_selection = 'training_crops_human_selection'
if os.path.exists(traning_dataset_folder_human_selection):
    shutil.rmtree(traning_dataset_folder_human_selection)
Path(traning_dataset_folder_human_selection).mkdir(parents=True, exist_ok=True)
for i in range(len(list_crops)):
    if flag_vector_ground_truth_all_true[i] == True:
        z = list_crops[i]
        im = Image.fromarray((z).astype(np.uint8))  # Normalize and convert to uint8
        im.save(f'{traning_dataset_folder_human_selection}/particle_{i}.png')
for i in range(len(list_crops)):
    if flag_vector_ground_truth_all_false[i] == True:
        z = list_crops[i]
        im = Image.fromarray((z).astype(np.uint8))  # Normalize and convert to uint8
        im.save(f'{traning_dataset_folder_human_selection}/no_particle_{i}.png')

In [ ]:
load_model_from_file = False
model_human_selected_data_path = 'particle_detection_cnn_human_selected_data.pth'
if load_model_from_file == False:
    importlib.reload(ML)
    model_human_selected_data, training_losses_human_selected_data, validation_losses_human_selected_data = ML.run_network(image_dir=traning_dataset_folder_human_selection, num_epochs=num_epochs,learning_rate = learning_rate,batch_size = batch_size)
    ML.save_model(model_human_selected_data, path=model_human_selected_data_path)
    plt.figure(figsize=(12, 6))
    plt.plot(training_losses_human_selected_data, label='Training Loss', color='blue', marker='o', linestyle='-', linewidth=2, markersize=6)
    plt.plot(validation_losses_human_selected_data, label='Validation Loss', color='red', marker='x', linestyle='--', linewidth=2, markersize=8)
    plt.title('Training and Validation Loss Over Epochs', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()  # Adjusts plot margins to give a better layout
    plt.show()
else:
    model_human_selected_data = ML.ParticleDetectionCNN()
    ML.load_model(model_human_selected_data, model_human_selected_data_path)

## Implementing the model with testing data 
____

In [ ]:
# # loading a list of crops 

# max_crops_to_display =50
# selected_time_point = None
# croparray_filtered, mean_crop_filtered, first_appearance, crop_size = AM.CropArray(image=filtered_image_normalized, df_crops=df_tracking, crop_size=crop_size, remove_outliers=False, max_percentile=99.9,selected_time_point=selected_time_point).run()
# list_crops = AM.Utilities().normalize_crop_return_list(array_crops_YXC=mean_crop_filtered,crop_size=crop_size,selected_color_channel=0,normalize_to_255=True)


### Testing SNR method
___

In [ ]:
number_crops = first_appearance.shape[0]//crop_size
flag_vector_SNR = np.zeros(number_crops, dtype=bool)
for crop_id in range(number_crops):
    flag_vector_SNR[crop_id]= AM.Utilities().is_spot_in_crop(crop_id, crop_size=crop_size, selected_color_channel=0, array_crops_YXC=mean_crop_filtered,show_plot=False,snr_threshold=1.1)
print('Number of detected spots SNR method:', np.sum(flag_vector_SNR))
AM.Plots().plot_average_crops (mean_crop_filtered, crop_size, plot_orientation='horizontal',show_particle_labels=True,save_plots=False,plot_name=None,max_crops_to_display=max_crops_to_display,flag_vector=flag_vector_SNR)

### Testing ML method with Real data
____

In [ ]:
importlib.reload(ML)
selected_color_channel = 0
flag_vector_ML_real = ML.predict_crops(model_real_data, list_crops,threshold=0.67)
print("Number of predicted particles ML method:", np.sum(flag_vector_ML_real))
AM.Plots().plot_average_crops (mean_crop_filtered, crop_size, plot_orientation='horizontal',show_particle_labels=True,save_plots=False,plot_name=None,max_crops_to_display=max_crops_to_display,flag_vector=flag_vector_ML_real)


### Testing ML method with simulated data

___

In [ ]:
importlib.reload(ML)
selected_color_channel = 0
flag_vector_ML_simulated = ML.predict_crops(model_simulated_data, list_crops,threshold=0.67)
print("Number of predicted particles ML method:", np.sum(flag_vector_ML_simulated))
AM.Plots().plot_average_crops (mean_crop_filtered, crop_size, plot_orientation='horizontal',show_particle_labels=True,save_plots=False,plot_name=None,max_crops_to_display=max_crops_to_display,flag_vector=flag_vector_ML_simulated)

### Testing ML method with human selected data
____

In [ ]:
importlib.reload(ML)
selected_color_channel = 0
flag_vector_ML_human_selected_data = ML.predict_crops(model_human_selected_data, list_crops,threshold=0.6)
print("Number of predicted particles ML method human selected data:", np.sum(flag_vector_ML_human_selected_data))
AM.Plots().plot_average_crops (mean_crop_filtered, crop_size, plot_orientation='horizontal',show_particle_labels=True,save_plots=False,plot_name=None,max_crops_to_display=max_crops_to_display,flag_vector=flag_vector_ML_human_selected_data)

In [ ]:
# impot the human selection
a = np.load('human_selection_1.npy')
b = np.load('human_selection_2.npy')
c = np.load('human_selection_3.npy')
d = np.load('human_selection_4.npy')

# all true
flag_vector_ground_truth_all_true = np.logical_and.reduce([a, b, c,d])
# all false
flag_vector_ground_truth_all_false = np.logical_not(np.logical_or.reduce([a, b, c,d]))
# flag at least two true
flag_vector_ground_truth_at_least_two_true = (a.astype(int) + b.astype(int) + c.astype(int) + d.astype(int)) >= 2

# List of boolean arrays
human_selection_arrays = [a, b, c, d]
flag_vector_consensus = np.sum(human_selection_arrays, axis=0) >= (len(human_selection_arrays) / 2)

# number of flag_vector_ground_truth_all_false
number_of_true = np.sum(flag_vector_consensus)
# sume the inverse of the human selection arrays
number_of_false = np.sum(np.logical_not(flag_vector_consensus), axis=0)
#number_of_true = np.sum(human_selection_arrays)
print('Number of true:', number_of_true)
print('Number of false:', number_of_false)

In [ ]:
importlib.reload(ML)
selected_color_channel = 0
print("Number of predicted particles ML method:", np.sum(flag_vector_consensus))
AM.Plots().plot_average_crops (mean_crop_filtered, crop_size, plot_orientation='horizontal',show_particle_labels=True,save_plots=False,plot_name=None,max_crops_to_display=120,flag_vector=flag_vector_consensus)


In [ ]:
def calculate_performance(predicted, ground_truth):
    TP = np.sum((predicted == True) & (ground_truth == True))
    FP = np.sum((predicted == True) & (ground_truth == False))
    TN = np.sum((predicted == False) & (ground_truth == False))
    FN = np.sum((predicted == False) & (ground_truth == True))
    # Calculate accuracy
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    return TP, FP, TN, FN, accuracy

# Calculate for SNR method
TP_SNR, FP_SNR, TN_SNR, FN_SNR, accuracy_SNR = calculate_performance(flag_vector_SNR, flag_vector_consensus)
print(f"SNR Method - Accuracy: {accuracy_SNR:.2%} (TP: {TP_SNR}, FP: {FP_SNR}, TN: {TN_SNR}, FN: {FN_SNR})")

# Calculate for ML method
TP_ML, FP_ML, TN_ML, FN_ML, accuracy_ML = calculate_performance(flag_vector_ML_real, flag_vector_consensus)
print(f"ML Method - Real Data - Accuracy: {accuracy_ML:.2%} (TP: {TP_ML}, FP: {FP_ML}, TN: {TN_ML}, FN: {FN_ML})")

# Calculate for ML method
TP_ML, FP_ML, TN_ML, FN_ML, accuracy_ML = calculate_performance(flag_vector_ML_simulated, flag_vector_consensus)
print(f"ML Method- Simulated Data - Accuracy: {accuracy_ML:.2%} (TP: {TP_ML}, FP: {FP_ML}, TN: {TN_ML}, FN: {FN_ML})")

# Calculate for ML method human selected data
TP_ML, FP_ML, TN_ML, FN_ML, accuracy_ML = calculate_performance(flag_vector_ML_human_selected_data, flag_vector_consensus)
print(f"ML Method- Human Selected Data - Accuracy: {accuracy_ML:.2%} (TP: {TP_ML}, FP: {FP_ML}, TN: {TN_ML}, FN: {FN_ML})")

In [ ]:
# Calculate Pearson correlation coefficient between the arrays
corr_ab = np.corrcoef(a, b)[0, 1]
corr_ac = np.corrcoef(a, c)[0, 1]
corr_bc = np.corrcoef(b, c)[0, 1]
corr_ad = np.corrcoef(a, d)[0, 1]

print(f"Correlation between 1 and 2: {corr_ab}")
print(f"Correlation between 1 and 3: {corr_ac}")
print(f"Correlation between 2 and 3: {corr_bc}")
print(f"Correlation between 2 and 4: {corr_ad}")


In [ ]:


global_results = []
def create_crop_widget(list_crops):
    number_crops = len(list_crops)
    current_crop_id = 0
    RESHAPE_IMAGE_SIZE = 32
    button_box = widgets.HBox()  # This will hold our buttons
    output = widgets.Output()
    info_label = widgets.Label()  # Label to display crop info
    
    def update_info_label():
        info_label.value = f"Spot {current_crop_id + 1}/{number_crops}"
    
    def display_crop(crop_id):
        crop = list_crops[crop_id]
        # transform crop 


        crop = np.array(Image.fromarray(crop).resize((RESHAPE_IMAGE_SIZE, RESHAPE_IMAGE_SIZE)))
        # min max normalization
        crop = (crop - np.min(crop)) / (np.max(crop) - np.min(crop))
        # remove the 0.1 and 99.9 percentile
        crop = crop - np.percentile(crop, 0.01)
        crop = crop / np.percentile(crop, 99.95)
        #crop = AM.RemoveExtrema(crop, min_percentile=0.5, max_percentile=99.5).remove_outliers() 
        # normalize to 255
        with output:
            clear_output(wait=True)
            update_info_label()
            plt.imshow(crop, cmap='gray')
            plt.axis('off')
            plt.show()

    def on_button_clicked(is_spot):
        # Append or replace the result in the global list
        if len(global_results) > current_crop_id:
            global_results[current_crop_id] = is_spot
        else:
            global_results.append(is_spot)
        move_next()

    def move_next():
        nonlocal current_crop_id
        current_crop_id += 1
        if current_crop_id < number_crops:
            display_crop(current_crop_id)
        else:
            # Show final message when done
            with output:
                clear_output(wait=True)
                print("No more crops to display.")
                print("Results:", global_results)

    def move_back():
        nonlocal current_crop_id
        if current_crop_id > 0:
            current_crop_id -= 1
            display_crop(current_crop_id)

    # Creating buttons for user interaction
    spot_present_button = widgets.Button(description='Spot Present', button_style='success')
    no_spot_button = widgets.Button(description='No Spot Present', button_style='danger')
    back_button = widgets.Button(description='Back', button_style='info')

    spot_present_button.on_click(lambda b: on_button_clicked(True))
    no_spot_button.on_click(lambda b: on_button_clicked(False))
    back_button.on_click(lambda b: move_back())

    button_box.children = [back_button, spot_present_button, no_spot_button, info_label]

    # Initial display setup
    display(button_box, output)
    display_crop(current_crop_id)

In [ ]:
#create_crop_widget(list_crops)

In [ ]:
human_selection = np.array(global_results)
human_selection = np.pad(human_selection, (0, len(list_crops) - len(human_selection)), 'constant', constant_values=(0, 0))
# save the human selection
#np.save('human_selection_Nate.npy', human_selection)